In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import operator
import random
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error as mse
import itertools
%matplotlib inline 

## Graph Creation

Create a graph named facebook from the Facebook data in file facebook-links.txt. As noted, we use the Graph class.

In [2]:
facebook = nx.Graph()

## Read dataset, Draw the Graph

File facebook-links.txt contains a list of all of the user-to-user links from the Facebook New Orleans networks. 
These links are undirected on Facebook.

Format: Each line contains two numeric user identifiers, 
meaning the second user appeared in the first user's friend list, 
and the first user appeared in the second user's friend list. 
Finally, the third column is a UNIX timestamp with the time of link establishment 
(if it could be determined, otherwise it is unavailable).

In [3]:
def open_source_file(filename):
    with open (filename,"r") as file:
        if filename == "../data/facebook-links.txt":
            content = file.readlines()
            for i in range(len(content)):
                point1 = content[i].split("\t")[0]
                if point1 not in facebook:
                    facebook.add_node(point1)
                point2 = content[i].split("\t")[1]
                if point2 not in facebook:
                    facebook.add_node(point2)
                facebook.add_edge(point1,point2)
        elif filename == "../data/facebook-links.txt":
            result = [x.strip("\n") for x in file.readlines()]
            for i in range(len(result)):
                point1 = result[i].split(" ")[0]
                if point1 not in facebook:
                    facebook.add_node(point1)
                point2 = result[i].split(" ")[1]
                if point2 not in facebook:
                    facebook.add_node(point2)
                facebook.add_edge(point1,point2)
        else:
            print ("Not a valid file, please check again")

In [4]:
open_source_file("../data/facebook-links.txt")

# Exploratory data analysis

## Graph basic information

In [5]:
def show_basic_info(graph):
    print (nx.info(graph))

In [6]:
show_basic_info(facebook)

Name: 
Type: Graph
Number of nodes: 63731
Number of edges: 817090
Average degree:  25.6418


for test purposes, print only first 10 edges of facebook graph

In [7]:
fbnodes = facebook.nodes()
print (fbnodes[: 10])

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


for test purposes, print only first 10 edges of facebook graph

In [8]:
fbedges = facebook.edges()
print(fbedges[: 10])

[('1', '2'), ('1', '3'), ('1', '4'), ('1', '5'), ('1', '6'), ('1', '7'), ('1', '8'), ('1', '9'), ('1', '10'), ('1', '11')]


In [9]:
adjacency_matrix = {}
num_of_friends = {}

for node in nx.nodes_iter(facebook):
    adjacency_matrix[node] = [neighbor for neighbor in nx.all_neighbors(facebook,node)]

for key in adjacency_matrix:
    num_of_friends[int(key)] = len(adjacency_matrix[key])
    
updat_num_of_friends = sorted(num_of_friends.items(), key=lambda x: (-x[1], x[0]))

In [10]:
num_users = len(adjacency_matrix)

utility = np.zeros((num_users, num_users))

print(num_users)
for user_id, user_friend_ids in adjacency_matrix.items():
    for x in user_friend_ids:
        utility[(int)(user_id) - 1, (int)(x) - 1] = 1
        
# should match above!
sparsity = float(len(utility.nonzero()[0]))
sparsity /= (utility.shape[0] * utility.shape[1])
sparsity *= 100
print('Sparsity: {:.2f}%'.format(sparsity))

63731
Sparsity: 0.04%


## Evaluation via Mean Squared Error (MSE)

In [11]:
def mse_utility(u1, u2):
    return mse(u1[u1.nonzero()].flatten(), u2[u2.nonzero()].flatten())

# u1 = np.array([[2, 0, 0, 4, 4], [5, 5, 5, 3, 3], [2, 4, 2, 1, 2]])
# u2 = np.array([[3, 0, 0, 4, 4], [5, 5, 5, 3, 3], [2, 4, 2, 1, 7]])

# print("{:.4f}".format(mse_utility(u1, u2)))

## Similarity via Cosine Distance

In [12]:
def cosine_sim(v1, v2):
    numerator = sum([x * y for x, y in zip(v1, v2)])
    denominator = np.sqrt(sum([x ** 2 for x in v1])) * np.sqrt(sum([x ** 2 for x in v2]))
    return numerator / denominator

In [13]:
def sim_matrix(u, eps=1.0e-9):
    step1 = u.dot(u.T) + eps
    step2 = np.array([np.sqrt(np.diagonal(step1))])
    return (step1 / step2 / step2.T)

In [14]:
%timeit -n 10 -r 3 sim_matrix(utility[:50,:])

10 loops, best of 3: 12 ms per loop


In [15]:
def sim_users(u):
    return sim_matrix(u)

print(sim_users(utility[:50,:50]))

[[  1.00000000e+00   5.58156306e-01   5.91312396e-01 ...,   7.41249317e-02
    1.96116135e-10   2.77350098e-01]
 [  5.58156306e-01   1.00000000e+00   6.67423813e-01 ...,   1.19522861e-01
    3.16227766e-10   2.23606798e-01]
 [  5.91312396e-01   6.67423813e-01   1.00000000e+00 ...,   1.13960576e-10
    3.01511344e-10   2.13200717e-01]
 ..., 
 [  7.41249317e-02   1.19522861e-01   1.13960576e-10 ...,   1.00000000e+00
    3.77964473e-01   2.67261242e-10]
 [  1.96116135e-10   3.16227766e-10   3.01511344e-10 ...,   3.77964473e-01
    1.00000000e+00   7.07106781e-10]
 [  2.77350098e-01   2.23606798e-01   2.13200717e-01 ...,   2.67261242e-10
    7.07106781e-10   1.00000000e+00]]


## K-Neighborhood

In [16]:
def top_k(arr, self_idx, k):
    val_index = { v:key for key, v in enumerate(arr) }
    top_k_val = sorted(val_index.keys())[::-1]
    i = 0
    res = {}
    while i < k:
        if val_index[top_k_val[i]] == self_idx:
            i += 1
            k += 1
            continue
        res[val_index[top_k_val[i]]] = top_k_val[i]
        i += 1
    return res

## Recommend via Similar Users

In [ ]:
def rec_via_users(m_utility, m_sim_users, user_idx, frd_idx, k):
    items = m_utility[:, frd_idx]
    i_sim = top_k(m_sim_users[:, user_idx], user_idx, k)
    non_zero_index = [i for i in i_sim if items[i] != 0]
    if sum([i_sim[i] for i in non_zero_index]) == 0:
        return 0
    return sum([i_sim[i] * items[i] for i in non_zero_index]) / sum([i_sim[i] for i in non_zero_index])

## Evaluation

In [ ]:
random.seed(12345)

def recs_via_users(m_utility, m_sim_users, k, test_n):
    test = random.sample(range(m_sim_users.shape[0]), test_n)
    true = []
    pred = []
    for user_idx in test:
        for item_idx in range(m_utility.shape[1]):
            if m_utility[user_idx][item_idx] != 0:
                true.append(m_utility[user_idx][item_idx])
                
                p = round(rec_via_users(m_utility, m_sim_users, user_idx, item_idx, k))
                if p != 0:    
                    pred.append(p)
                else:
                    pred.append(1.0e-9)
                        

    return mse_utility(np.array([true], dtype=np.float64), np.array([pred], dtype=np.float64))
    
similarity_users = sim_users(utility)

ks = []
mses = []
for i in range(50):
    ks.append(i+1)
    mses.append(recs_via_users(utility, similarity_users, i+1, 100))
    print("{}/50".format(i+1), mses[-1])

In [ ]:
friends_num = pd.DataFrame(data=updat_num_of_friends)
friends_num.columns = ['number of friends', 'user_id']
friends_num.head()

## friends set 

Returns a set of the friends of the given user, in the given graph.
The parameter 'user' is the string name of a person in the graph.

In [ ]:
def friends(graph, user):
    return set(graph.neighbors(user))

## friends of friends set

Returns a set of friends of friends of the given user, in the given graph.
The result does not include the given user nor any of that user's friends.

In [ ]:
def friends_of_friends(graph, user):
    f_list = set()
    friend_list = friends(graph,user)
    for friend in friend_list:
        f_list.update(set(friends(graph,friend)))
    for friend in friend_list:
        f_list.discard(friend)
    f_list.remove(user)
    return f_list

## Common friends between two users

Returns the set of friends that user1 and user2 have in common.

In [ ]:
def common_friends(graph, user1, user2):
    friend_set_1 = friends(graph,user1)
    friend_set_2 = friends(graph,user2)
    return (friend_set_1 & friend_set_2)

Returns a map from each user U to the number of friends U has in common with the given user.
    The map keys are the users who have at least one friend in common with the
    given user, and are neither the given user nor one of the given user's friends.
    Take a graph G for example:
        - A and B have two friends in common
        - A and C have one friend in common
        - A and D have one friend in common
        - A and E have no friends in common
        - A is friends with D
    number_of_common_friends_map(G, "A")  =>   { 'B':2, 'C':1 }

In [ ]:
def number_of_common_friends_map(graph, user):
    number_common = { str(friend) : len(common_friends(graph,str(friend),user)) for friend in friends_of_friends(graph,user) }
    return number_common

## Key list

Given a map whose values are numbers, return a list of the keys. The keys are sorted by the number they map to, from greatest to least. When two keys map to the same number, the keys are sorted by their natural sort order, from least to greatest.

In [ ]:
def number_map_to_sorted_list(map):
    res = sorted(map.items(), key=lambda x: (-x[1], x[0]))
    numbers = [item[0] for item in res]
    return numbers

## Recommend  by number of common friends

Return a list of friend recommendations for the given user.The friend recommendation list consists of names of people in the graph who are not yet a friend of the given user.The order of the list is determined by the number of common friends.

In [ ]:
def recommend_by_number_of_common_friends(graph, user):
    common_friends = number_of_common_friends_map(graph,user)
    return number_map_to_sorted_list(common_friends)

## Influence scoring

Returns a map from each user U to the friend influence, with respect to the given user. The map only contains users who have at least one friend in common with U, and are neither U nor one of U's friends.

"Influence scoring": the score for user2 as a friend of user1 is: 1/numfriends(f1) + 1/numfriends(f2) + 1/numfriends(f3), where numfriends(f) is the number of friends that f has. In other words, each friend F of user1 has a total influence score of 1 to contribute, and divides it equally among all of F's friends.

In [ ]:
def influence_map(graph, user):
    fre_of_fre = friends_of_friends(graph,user)
    com_fre = { str(friend) : common_friends(graph,friend,user) for  friend in fre_of_fre}
    influence = { str(friend) : sum([1/len(friends(graph,item)) for item in common_friend]) for friend, common_friend in com_fre.items() }
    return influence

## Recommend by influence

Return a list of friend recommendations for the given user. The friend recommendation list consists of names of people in the graph who are not yet a friend of the given user. The order of the list is determined by the influence measurement.

In [ ]:
def recommend_by_influence(graph, user):
    friends_influence = influence_map(graph,user)
    res = number_map_to_sorted_list(friends_influence)
    return res

For every Facebook user with an id that is a multiple of 100, 
print a list containing the first 10 friend recommendations, 
as determined by number of common friends. 
If there are fewer than 10 recommendations, 
print all the recommendations.

In [ ]:
rec_num_of_friends = []
for i in range(len(facebook.nodes())):
    if ((i%100 == 0) and (str(i) in facebook)):
        rec_num_of_friends.append(recommend_by_number_of_common_friends(facebook,str(i))[:10])

In [ ]:
rec_num_of_friends

For every Facebook user with an id that is a multiple of 1000, 
print a list containing the first 10 friend recommendations, 
as determined by influence score. 
If there are fewer than 10 recommendations, 
print all the recommendations.

In [ ]:
rec_influence = []
for i in range(len(facebook.nodes())):
    if ((i%1000 == 0) and (str(i) in facebook)):
        rec_influence.append(recommend_by_influence(facebook,str(i))[:10])

In [ ]:
rec_influence

Present the average index for each recommendation system. 
State which recommendation system is better for the facebook graph.

## Evaluation

In [ ]:
def evaluate_recommendation(graph):
    number_index = []
    influence_index = []

    for i in range(100):
        # 1. Randomly choose a real friend connection; call the two friends F1 and F2.
        friendship_chosen = random.choice(graph.edges())
        friend1 = friendship_chosen[0]
        friend2 = friendship_chosen[1]

        # 2. Remove their friendship from the graph.
        graph.remove_edge(friend1,friend2)

        '''
        3. Compute friend recommendations for F1 and F2.
        4. Determine the rank of F1 in F2's list of recommended friends.
            Determine the rank of F2 in F1's list of recommended friends.
            If either of these does not exist (e.g., F1 is not recommended as one of F2's friends), discard the F1-F2 pair from your experiment.
            Otherwise, average these two numbers.
            The "rank" is also known as the "index" or "position". It starts counting at 1, not 0.
        '''
        if ((len(graph.neighbors(friend1)) == 0) or (len(graph.neighbors(friend2)) == 0)):
            pass
        else:
            f1_rec_number = recommend_by_number_of_common_friends(graph,friend1)
            f2_rec_number = recommend_by_number_of_common_friends(graph,friend2)
        
            f1_rec_influence = recommend_by_influence(graph,friend1)
            f2_rec_influence = recommend_by_influence(graph,friend2)

            if ((friend2 not in f1_rec_number) or (friend1 not in f2_rec_number) or (friend2 not in f1_rec_influence) or (friend1 not in f2_rec_influence)):
                pass
            else:
                # recommend friend by the number of common friends
                index_number_f1= f1_rec_number.index(friend2) + 1
                number_index.append(index_number_f1)

                index_number_f2= f2_rec_number.index(friend1) + 1
                number_index.append(index_number_f2)
               
                # recommend friend by friends' influence
                index_influence_f1 = f1_rec_influence.index(friend2) + 1
                influence_index.append(index_influence_f1)

                index_influence_f2 = f2_rec_influence.index(friend1) + 1
                influence_index.append(index_influence_f2)

        #5. Put their friendship back in the graph.
        graph.add_edge(friend1,friend2)
            
    sum_influence_index = sum([influence_index[i] for i in range(len(influence_index))])
    avg_influence = sum_influence_index / len(influence_index)
    print("Average rank of influence method:", avg_influence)

    # calculate the average of number of common friends
    # calculate the average of number of common friends
    sum_number_index = sum([number_index[i] for i in range(len(number_index))])
    avg_number = sum_number_index/len(number_index)  
    print ("Average rank of number of friends in common method:", avg_number)

    # compare two methods
    if (avg_influence < avg_number):
        print ("recommend by influence is better")
    else:
        print ("recommend by number of friends in common method is better")

In [ ]:
evaluate_recommendation(facebook)